# Brain Tumor Segmentation — DeepLabV3 Transfer Learning (v2)

This notebook fine-tunes a pretrained DeepLabV3 (ResNet-50) model from torchvision for binary brain tumor segmentation on single-channel (grayscale) MRI slices.

Primary goals:

- Adapt a pretrained DeepLabV3 model to accept 1-channel input and output two classes (background vs tumor).
- Freeze the ResNet backbone and fine-tune the segmentation heads to leverage pretrained features while reducing training time.
- Create a robust PyTorch Dataset and DataLoaders for grayscale images and binary masks.
- Use Dice loss and MONAI's Dice metric for segmentation-specific optimization and evaluation. Automatic Mixed Precision (AMP) is enabled for faster GPU training.

Assumptions:

- Input images are stored as PNG files in `brain_tumor_dataset/images` and masks in `brain_tumor_dataset/masks`.
- Masks are binary or use 0/255; the dataset class binarizes masks automatically.


In [1]:
import torch
import torchvision


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device, torch.__version__, torchvision.__version__

(device(type='cuda'), '2.8.0+cu126', '0.23.0+cu126')

## 1. Load the pretrained model

We load DeepLabV3 (ResNet-50) with torchvision's DEFAULT weights to leverage pretrained feature extractors. The weights object provides a `transforms()` helper describing the original model's expected preprocessing.

Because our MRI slices are grayscale, we build a custom transform pipeline that:

- Resizes images to the network's expected resolution,
- Converts RGB to a single grayscale channel,
- Converts to a tensor and normalizes with a 1-channel mean/std.

After loading the pretrained model we will:

1. Freeze the backbone parameters to keep pretrained features fixed during initial training.
2. Replace the classifier and auxiliary classifier to output 2 channels (background vs tumor).
3. Update the first convolution to accept a single input channel.

This approach reduces the amount of training required while adapting the architecture to our data.


In [2]:
weights = torchvision.models.segmentation.DeepLabV3_ResNet50_Weights.DEFAULT
transforms = weights.transforms()

deeplap_V3 = torchvision.models.segmentation.deeplabv3_resnet50(weights = weights)
deeplap_V3

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [3]:
from torchinfo import summary

summary(
    deeplap_V3,
    # The input size should match the model's 2D configuration: (batch_size, channels, height, width)
    input_size=(1, 3, 224, 224),
    col_names=("input_size", "output_size", "num_params", "trainable"),
)

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Trainable
DeepLabV3                                          [1, 3, 224, 224]          [1, 21, 224, 224]         --                        True
├─IntermediateLayerGetter: 1-1                     [1, 3, 224, 224]          [1, 2048, 28, 28]         --                        True
│    └─Conv2d: 2-1                                 [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     True
│    └─BatchNorm2d: 2-2                            [1, 64, 112, 112]         [1, 64, 112, 112]         128                       True
│    └─ReLU: 2-3                                   [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --
│    └─MaxPool2d: 2-4                              [1, 64, 112, 112]         [1, 64, 56, 56]           --                        --
│    └─Sequential: 2-5                             [1, 64, 56

In [4]:
transforms # notice how the mean and std are on 3 channels, but we need only one

SemanticSegmentation(
    resize_size=[520]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [5]:
# Define the transform for single-channel (grayscale) images
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode

deeplab_V3_transform = T.Compose([
    T.Resize(520, interpolation=InterpolationMode.BILINEAR),
    T.Grayscale(num_output_channels=1),   # convert to 1 channel
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5])    # adjust mean/std for 1 channel
])

In [6]:
from torchinfo import summary

# Freeze backbone
for param in deeplap_V3.backbone.parameters():
    param.requires_grad = False
    
    
# Change classifier and auxiliary head to 2 only
deeplap_V3.classifier[4] =  torch.nn.Conv2d(256, 2, # this is the only change
                                            kernel_size=(1, 1), stride=(1, 1))
deeplap_V3.aux_classifier[4] =  torch.nn.Conv2d(256, 2, # this is the only change
                                                kernel_size=(1, 1), stride=(1, 1))

# Change the input to be 1 channel only (not 3)
deeplap_V3.backbone['conv1'] = torch.nn.Conv2d (1 # this is the only change
                                                , 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

summary(
    deeplap_V3,
    # The input size should match the model's 2D configuration: (batch_size, channels, height, width)
    input_size=(1, 1, 224, 224),
    col_names=("input_size", "output_size", "num_params", "trainable"),
)

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Trainable
DeepLabV3                                          [1, 1, 224, 224]          [1, 2, 224, 224]          --                        Partial
├─IntermediateLayerGetter: 1-1                     [1, 1, 224, 224]          [1, 2048, 28, 28]         --                        Partial
│    └─Conv2d: 2-1                                 [1, 1, 224, 224]          [1, 64, 112, 112]         3,136                     True
│    └─BatchNorm2d: 2-2                            [1, 64, 112, 112]         [1, 64, 112, 112]         (128)                     False
│    └─ReLU: 2-3                                   [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --
│    └─MaxPool2d: 2-4                              [1, 64, 112, 112]         [1, 64, 56, 56]           --                        --
│    └─Sequential: 2-5                             [1,

## 2. Create dataset and dataloaders

This section prepares the dataset and DataLoaders. Steps:

1. Implement a custom Dataset that reads grayscale images and masks, binarizes masks, and applies optional transforms (e.g., resizing/augmentation).
2. Instantiate the full dataset and split it into training and validation subsets (80/20 split).
3. Create DataLoaders with appropriate batch sizes and worker counts for training and validation.

Keep the transforms consistent between training and validation (except for any augmentations you only want during training).


### 2.1 Dataset details

The custom `BrainTumorDataset` does the following:

- Loads images and masks from the provided directories and sorts filenames to keep pairs aligned.
- Converts both image and mask to grayscale (`L` mode) and returns them as NumPy arrays.
- Binarizes masks by thresholding at 127 (so 0/255 masks become 0/1).
- If a transform is provided (e.g., Albumentations), it is applied to both image and mask together.

Important: Ensure the dataset directories contain matching image/mask pairs and are named consistently.


In [7]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class BrainTumorDataset(Dataset):
    """Custom PyTorch Dataset for loading brain tumor images and masks."""
    
    def __init__(self, image_dir, mask_dir, transform=None):
        """
        Args:
            image_dir (pathlib.Path or str): Directory with all the images.
            mask_dir (pathlib.Path or str): Directory with all the masks.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.transform = transform
        
        # Get all image and mask file paths and sort them to ensure they correspond
        self.images = sorted(list(image_dir.glob("*.png")))
        self.masks = sorted(list(mask_dir.glob("*.png")))
        
        # Ensure the number of images and masks are the same for data integrity
        assert len(self.images) == len(self.masks), "Number of images and masks must be equal."

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.images)

    def __getitem__(self, idx):
        """
        Fetches the sample at the given index, loads it, and applies transforms.

        Args:
            idx (int): The index of the sample to fetch.

        Returns:
            tuple: (image, mask) where image is the transformed image and mask is the transformed mask.
        """
        # Get the file paths for the image and mask
        image_path = self.images[idx]
        mask_path = self.masks[idx]
        
        # Open the image and mask files
        # Images and masks are converted to grayscale (L) for single-channel processing
        image = np.array(Image.open(image_path).convert("L"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        
        # Normalize mask to be 0 or 1, as pixel values are often 0 and 255
        # A more robust and explicit way to binarize the mask
        mask = (mask > 127).astype(np.float32)
        
        """
        (mask > 127): This creates a boolean array 
                    where pixels with a value greater than 127 become True 
                    and all others become False.
                    
        .astype(np.float32): This converts the boolean array into a float array, 
                            where True becomes 1.0 and False becomes 0.0.
        """

        
        # Apply transformations if they are provided
        if self.transform:
            # Most transform libraries like Albumentations expect a dictionary
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]
            
        return image, mask



In [8]:
# init datasets
from pathlib import Path
from torch.utils.data import random_split


IMAGES_DIR = Path("brain_tumor_dataset/images")
MASKS_DIR = Path("brain_tumor_dataset/masks")

# 1. Instantiate the full dataset
full_dataset = BrainTumorDataset(image_dir= IMAGES_DIR,
                            mask_dir= MASKS_DIR,
                            transform= deeplab_V3_transform)

# 2. Define split sizes (80% train, 20% validation)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# 3. Perform the split
generator = torch.Generator().manual_seed(42) # for reproducibility
train_dataset, val_dataset = random_split(full_dataset, 
                                        [train_size, val_size], 
                                        generator=generator)

# Test print lengths
print(f"Train dataset: {train_dataset}")
print(f"Validation dataset: {val_dataset}")

print(f"Total samples: {len(full_dataset)}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

Train dataset: <torch.utils.data.dataset.Subset object at 0x000002B245DB6F50>
Validation dataset: <torch.utils.data.dataset.Subset object at 0x000002B245DB4A30>
Total samples: 3064
Training samples: 2451
Validation samples: 613


### 2.2 DataLoaders

We create PyTorch DataLoaders for training and validation. Important considerations:

- Use `shuffle=True` for the training loader to randomize samples each epoch.
- Use reasonable `batch_size` values based on GPU memory.
- Set `num_workers` to a value appropriate for your machine (e.g., `os.cpu_count()`), and use `pin_memory=True` when training on CUDA.

The loaders return (image, mask) tuples ready for the training loop.


In [9]:
from torch.utils.data import DataLoader
import os

train_loader = DataLoader(dataset= train_dataset,
                        batch_size= 8,
                        shuffle= True,
                        pin_memory= True,
                        num_workers= os.cpu_count())

val_loader = DataLoader(dataset= val_dataset,
                        batch_size= 4,
                        shuffle= False,
                        pin_memory= True,
                        num_workers= os.cpu_count()//2)

train_loader, val_loader

(<torch.utils.data.dataloader.DataLoader at 0x2b245db5180>,
 <torch.utils.data.dataloader.DataLoader at 0x2b245db62c0>)

## 3. Loss function, optimizer, and metrics

Choices and rationale:

- Loss: MONAI's `DiceLoss(sigmoid=True)` is used because Dice is appropriate for segmentation and encourages overlap between predictions and ground truth. `sigmoid=True` applies a per-pixel sigmoid to raw logits.
- Optimizer: Adam with a small learning rate (1e-4) is a good default for fine-tuning segmentation heads.
- Scheduler: `ReduceLROnPlateau` reduces the learning rate if validation loss plateaus, helping escape local minima.
- Metric: MONAI's `DiceMetric` (with `include_background=False`) evaluates segmentation quality excluding background to focus on tumor detection.
- AMP: Automatic Mixed Precision via `torch.amp.GradScaler` speeds up training on modern GPUs and reduces memory usage.

We also keep a `best_val_dice` to checkpoint the best-performing model during training.


In [10]:
from monai.losses import DiceLoss
from monai.metrics import DiceMetric


# Define the loss function. DiceLoss is excellent for segmentation tasks.
# sigmoid=True is necessary as our model outputs raw logits. 
# It applies a sigmoid to the output before calculating loss.
loss_fn = DiceLoss(sigmoid=True)

# Define the optimizer.
optimizer = torch.optim.Adam(deeplap_V3.parameters(), 
                            lr=1e-4)

# Define a learning rate scheduler. 
# It will reduce the learning rate when the validation loss stops improving ('min' mode).
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                        mode='min', 
                                                        factor=0.1, 
                                                        patience=10)


# Initialize the Dice Metric for evaluating segmentation performance.
# `include_background=False` means the background class (label 0) is ignored during calculation.
# `reduction="mean"` specifies that the Dice scores for all classes will be averaged.
metric = DiceMetric(include_background=False,
                    reduction= "mean")

# Set the total number of training epochs.
epochs = 20

# Initialize a variable to track the best validation Dice score achieved so far.
# This is used to save the best performing model checkpoint.
best_val_dice = 0.0

# Initialize a Gradient Scaler for Automatic Mixed Precision (AMP) training.
# This helps prevent underflow of gradients when using float16 precision,
# improving training stability and speed on compatible GPUs.
scaler = torch.amp.GradScaler(device = "cuda")



## 4. Training loop

Training uses a modular engine (imported from `going_modular.engine`) to encapsulate the training and validation steps. Key points:

- The engine handles forward/backward passes, loss computation, metric updates, AMP scaling, scheduler steps, and checkpointing.
- Provide model, data loaders, loss function, optimizer, scheduler, scaler, device, number of epochs, and a checkpoint directory.
- Checkpointing saves the best model (by validation Dice) so you can resume or evaluate the best weights later.

After training, inspect the saved checkpoints and validation metrics to assess performance. Consider unfreezing parts of the backbone and fine-tuning with a lower learning rate if more improvement is needed.


In [ ]:
from going_modular import engine

results = engine.train(model = deeplap_V3,
                    train_loader = train_loader,
                    test_loader = val_loader, 
                    loss_fn = loss_fn, 
                    optimizer = optimizer, 
                    scheduler = scheduler,
                    scaler = scaler, 
                    device = "cuda" if torch.cuda.is_available() else "cpu", 
                    epochs = epochs,
                    checkpoint_dir = "checkpoints_2/ ")



===== Starting Epoch 1/20 =====


Train E-1:   0%|          | 0/307 [00:00<?, ?it/s]